# Lambda School Data Science Module 141
## Statistics, Probability, and Inference

## Prepare - examine what's available in SciPy

As we delve into statistics, we'll be using more libraries - in particular the [stats package from SciPy](https://docs.scipy.org/doc/scipy/reference/tutorial/stats.html).

In [1]:
from scipy import stats
dir(stats)

['__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_binned_statistic',
 '_constants',
 '_continuous_distns',
 '_discrete_distns',
 '_distn_infrastructure',
 '_distr_params',
 '_multivariate',
 '_stats',
 '_stats_mstats_common',
 '_tukeylambda_stats',
 'absolute_import',
 'alpha',
 'anderson',
 'anderson_ksamp',
 'anglit',
 'ansari',
 'arcsine',
 'argus',
 'bartlett',
 'bayes_mvs',
 'bernoulli',
 'beta',
 'betaprime',
 'binned_statistic',
 'binned_statistic_2d',
 'binned_statistic_dd',
 'binom',
 'binom_test',
 'boltzmann',
 'boxcox',
 'boxcox_llf',
 'boxcox_normmax',
 'boxcox_normplot',
 'bradford',
 'burr',
 'burr12',
 'cauchy',
 'chi',
 'chi2',
 'chi2_contingency',
 'chisquare',
 'circmean',
 'circstd',
 'circvar',
 'combine_pvalues',
 'contingency',
 'cosine',
 'crystalball',
 'cumfreq',
 'describe',
 'dgamma',
 'dirichlet',
 'distributions',
 'division',
 'dlaplace',
 'dweibull',
 'energy_dist

In [2]:
# As usual, lots of stuff here! There's our friend, the normal distribution
norm = stats.norm()
print(norm.mean())
print(norm.std())
print(norm.var())

0.0
1.0
1.0


In [3]:
# And a new friend - t
t1 = stats.t(5)  # 5 is df "shape" parameter
print(t1.mean())
print(t1.std())
print(t1.var())

0.0
1.2909944487358056
1.6666666666666667


![T distribution PDF with different shape parameters](https://upload.wikimedia.org/wikipedia/commons/4/41/Student_t_pdf.svg)

*(Picture from [Wikipedia](https://en.wikipedia.org/wiki/Student's_t-distribution#/media/File:Student_t_pdf.svg))*

The t-distribution is "normal-ish" - the larger the parameter (which reflects its degrees of freedom - more input data/features will increase it), the closer to true normal.

In [4]:
t2 = stats.t(30)  # Will be closer to normal
print(t2.mean())
print(t2.std())
print(t2.var())

0.0
1.0350983390135313
1.0714285714285714


Why is it different from normal? To better reflect the tendencies of small data and situations with unknown population standard deviation. In other words, the normal distribution is still the nice pure ideal in the limit (thanks to the central limit theorem), but the t-distribution is much more useful in many real-world situations.

History sidenote - this is "Student":

![William Sealy Gosset](https://upload.wikimedia.org/wikipedia/commons/4/42/William_Sealy_Gosset.jpg)

*(Picture from [Wikipedia](https://en.wikipedia.org/wiki/File:William_Sealy_Gosset.jpg))*

His real name is William Sealy Gosset, and he published under the pen name "Student" because he was not an academic. He was a brewer, working at Guinness and using trial and error to determine the best ways to yield barley. He's also proof that, even 100 years ago, you don't need official credentials to do real data science!

## Live Lecture - let's perform and interpret a t-test

We'll generate our own data, so we can know and alter the "ground truth" that the t-test should find. We will learn about p-values and how to interpret "statistical significance" based on the output of a hypothesis test.

In [0]:
# TODO - during class, but please help!

## Assignment - apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [1]:
import pandas as pd
import numpy as np
import scipy

In [3]:
congress_data_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data'

# shell out to open this dataset
!curl https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data

republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y
democrat,n,y,y,n,y,y,n,n,n,n,n,n,y,y,y,y
democrat,n,y,n,y,y,y,n,n,n,n,n,n,?,y,y,y
republican,n,y,n,y,y,y,n,n,n,n,n,n,y,y,?,y
republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,y
democrat,y,y,y,n,n,n,y,y,y,n,n,n,n,n,?,?
republican,n,y,n,y,y,n,n,n,n,n,?,?,y,y,n,n
republican,n,y,n,y,y,y,n,n,n,n,y,?,y,y,?,?
democrat,n,y,y,n,n,n,y,y,y,n,n,n,y,n,?,?
democrat,y,y,y,n,n,y,y,y,?,y,y,?,n,n,y,?
republican,n,y,n,y,y,y,n,n,n,n,n,y,?,?,n,?
republican,n,y,n,y,y,y,n,n,n,y,n,y,y,?,n,?
democrat,y,n,y,n,n,y,n,y,?,y,y,y,?,n,n,y
democrat,y,?,y,n,n,n,y,y,y,n,n,n,y,n,y,y
republican,n,y,n,y,y,y,n,n,n,n,n,?,y,y,n,n
democrat,y,y,y,n,n,n,y,y,y,n,y,n,n,n,y,y
democrat,y,y,y,n,n,?,y,y,n,n,y,n,n,n,y,y
democrat,y,y,y,n,n,n,y,y,y,n,n,n,?,?,y,y
democrat,y,?,y,n,n,n,y,y,y,n,n,?,n,n,y,y
democrat,y,y,y,n,n,n,y,y,y,n,n,n,n,n,y,

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 18171  100 18171    0     0  48456      0 --:--:-- --:--:-- --:--:-- 48456


In [23]:
# make dataframe and rename columns
col_names = ['party', 'iss_1', 'iss_2', 'iss_3', 'iss_4',
             'iss_5', 'iss_6', 'iss_7', 'iss_8', 'iss_9',
             'iss_10', 'iss_11', 'iss_12', 'iss_13',
             'iss_14', 'iss_15', 'iss_16']
congress_data = pd.read_csv(congress_data_url, header=None, names=col_names)
print(congress_data.shape)
congress_data.head()

(435, 17)


,party,iss_1,iss_2,iss_3,iss_4,iss_5,iss_6,iss_7,iss_8,iss_9,iss_10,iss_11,iss_12,iss_13,iss_14,iss_15,iss_16
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [27]:
# check for null values
congress_data = congress_data.replace('?', np.NaN)
congress_data.isna().sum()
congress_data.isna().sum().sum()

392

In [15]:
# potential total possible votes
shape = congress_data.shape
shape[0] * shape[1] - 1

7394

In [30]:
# copy congress_data to a working df
df = congress_data.copy()

In [88]:
# quickly explore total members of each party and total 'y' or 'n' votes
# for each party on each issue
df[df.party == 'republican'].count()
df[df.party == 'democrat'].count()  # so 267 Democrats, 168 Republicans


party     267
iss_1     258
iss_2     239
iss_3     260
iss_4     259
iss_5     255
iss_6     258
iss_7     259
iss_8     263
iss_9     248
iss_10    263
iss_11    255
iss_12    249
iss_13    252
iss_14    257
iss_15    251
iss_16    185
dtype: int64

In [86]:
# find what party 'y' votes were for issue 1
df.party[df.iss_1 == 'y'].value_counts()

democrat      156
republican     31
Name: party, dtype: int64

In [132]:
# iterate over each of the issues and get vote counts
party = ('democrat', 'republican')
for issue in df.columns[1:]:
    print(df[str(issue)].value_counts())

n    236
y    187
Name: iss_1, dtype: int64
y    195
n    192
Name: iss_2, dtype: int64
y    253
n    171
Name: iss_3, dtype: int64
n    247
y    177
Name: iss_4, dtype: int64
y    212
n    208
Name: iss_5, dtype: int64
y    272
n    152
Name: iss_6, dtype: int64
y    239
n    182
Name: iss_7, dtype: int64
y    242
n    178
Name: iss_8, dtype: int64
y    207
n    206
Name: iss_9, dtype: int64
y    216
n    212
Name: iss_10, dtype: int64
n    264
y    150
Name: iss_11, dtype: int64
n    233
y    171
Name: iss_12, dtype: int64
y    209
n    201
Name: iss_13, dtype: int64
y    248
n    170
Name: iss_14, dtype: int64
n    233
y    174
Name: iss_15, dtype: int64
y    269
n     62
Name: iss_16, dtype: int64


In [135]:
for issue in df.columns[1:]:
    print(pd.crosstab(df[str(issue)], df.party), '\n')

party  democrat  republican
iss_1                      
n           102         134
y           156          31 

party  democrat  republican
iss_2                      
n           119          73
y           120          75 

party  democrat  republican
iss_3                      
n            29         142
y           231          22 

party  democrat  republican
iss_4                      
n           245           2
y            14         163 

party  democrat  republican
iss_5                      
n           200           8
y            55         157 

party  democrat  republican
iss_6                      
n           135          17
y           123         149 

party  democrat  republican
iss_7                      
n            59         123
y           200          39 

party  democrat  republican
iss_8                      
n            45         133
y           218          24 

party  democrat  republican
iss_9                      
n            60         146
y   

In [ ]:
# constants [for this exercise, anyway]
DEMOS = 267
REPUBS = 168